In [105]:
import numpy as np
import pandas as pd
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

# Gensim
import spacy
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from pprint import pprint

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,f1_score,plot_confusion_matrix,make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
np.random.seed(42)

In [24]:
train_df = pd.read_csv('train.csv')

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38740 entries, 0 to 38739
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   oid       38740 non-null  int64 
 1   category  38740 non-null  object
 2   text      38740 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.1+ KB


In [26]:
df = train_df.copy()

Объединим тексты каждого пользователя

In [27]:
df['text'] = df.groupby(['oid'])['text'].transform(lambda x: ' '.join(x))

In [28]:
df.drop_duplicates(inplace=True)
df

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
18606,700566540,boardgames,А вот и очередная среда пришла а за не ворвалс...
19040,239478272,basketball,БАСКЕТБОЛЬНЫЕ КРОССЫ ПОКУПАЙ ЗДЕСЬ tokenoidtok...
19738,819110546,martial_arts,Самураи вытаскиваю стрелы из лица своего товар...
21357,886287175,volleyball,Хотелось бы предложить вашему вниманию ролку в...


In [29]:
df['category'].value_counts()

autosport       316
extreme         311
martial_arts    305
motosport       303
boardgames      302
tennis          300
esport          299
athletics       297
hockey          295
volleyball      295
football        286
basketball      285
winter_sport    280
Name: category, dtype: int64

In [15]:
cat = {'winter_sport': 0, 'extreme': 1, 'football': 2, 'boardgames': 3,
       'hockey': 4, 'esport': 5, 'athletics': 6, 'motosport': 7, 'basketball': 8,
       'tennis': 9, 'autosport': 10, 'martial_arts': 11, 'volleyball': 12}
inv_cat = {v: k for k, v in cat.items()}

In [30]:
df.replace({'category':cat}, inplace= True)

In [31]:
russian_stopwords = stopwords.words("russian")
russian_stopwords += ['это', 'очень', 'сегодня', 'также', 'который', 'просто',
                      'всем', 'нужно', 'будет', 'будут', 'нам', 'поэтому']
nlp = spacy.load('ru_core_news_sm')

In [32]:
def remove_emoji(string):
    """
    Удаление эмоджи из текста
    """
    emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_extra_marks(abstract):
    "Return the abstracts after remvoing extra marks"
    extra_keys = ["’","—","”","“","-","–"]
    abstract_tokens = word_tokenize(abstract) 
    filtered_abstract = [word for word in abstract_tokens if not word in extra_keys] 
    return " ".join(filtered_abstract)

def replace_symbols(text,dic):
    "Return the abstracts after remvoing symbols in dic"
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def remove_stopwords(text, stop_words = russian_stopwords):
    "Return the text after remvoing stopwords"
    article_tokens = word_tokenize(text) 
    filtered_article = [word for word in article_tokens if not word in stop_words] 
    return " ".join(filtered_article)

def remove_ascii(text):
    for i in text:
        if 33 < ord(i) < 128:
            text = text.replace(i,'')
    return text.strip()


def lemmatize(abstract):
    """Return text after performing the lemmatization"""
    doc = nlp(abstract)
    #tokens = [token for token in doc]
    return  " ".join([token.lemma_ for token in doc])

def split_capital(text):
    text = text.split()
    res = ''
    for i in text:
        x = re.findall(r'[A-ZА-Я][a-zа-я]+', i)
        res += ' '.join(x) + ' ' if x else i + ' '
    return res.strip()


def preprocessing(abstracts):
    "Make abstracts preprocessing"
    d = { '\n': "", '\t': "", '\xa0':' '}
    abstracts = abstracts.apply(lambda x: replace_symbols(x,d))
    # remove emoji
    abstracts = abstracts.apply(remove_emoji)
    # lowcase
    #abstracts = abstracts.apply(lambda t : t.lower())
    #removing punctuations 
    abstracts = abstracts.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
    # remove formulas like $...$
    abstracts = abstracts.apply(lambda x: re.sub(r'\$.+\$', '', x)) 
    #removing digits
    abstracts = abstracts.apply(lambda x: re.sub('\w*\d\w*','', x))
    #removing extra marks
    abstracts = abstracts.apply(remove_extra_marks)
    #removing words with length <=2
    abstracts = abstracts.apply(lambda x: re.compile(r'\W*\b\w{1,2}\b').sub('', x))
    #remove latin symbols
    #abstracts = abstracts.apply(remove_ascii)
    #lemmatization
    #abstracts = abstracts.apply(lemmatize)
    #removing stopwords
    #abstracts = abstracts.apply(remove_stopwords)
    abstracts = abstracts.apply(split_capital)
    return abstracts

In [33]:
%%time
df['text']=preprocessing(df['text'])

Wall time: 8.91 s


In [79]:
df.reset_index(inplace=True)

In [42]:
from nltk.tokenize import word_tokenize

def tokenize(text):
    return word_tokenize(text)


def remove_token_words(text:list):
    new_text=[]
    for i in text:
        if 'token' not in i:
            new_text.append(i)
    return new_text


def lower_text(text:list):
    return [x.lower() for x in text]


tokens = df['text'].apply(tokenize)
for i in range(len(tokens)):
    tokens[i] = remove_token_words(tokens[i])
for i in range(len(tokens)):
    tokens[i] = lower_text(tokens[i])

In [43]:
docs = []
for i in tokens:
    docs.append(" ".join(i))

In [46]:
df['clean_text'] = docs

In [45]:
%%time
#removing stopwords
df['clean_text'] = df['clean_text'].apply(remove_stopwords)
#lemmatization
df['clean_text'] = df['clean_text'].apply(lemmatize)

Wall time: 7min 4s


In [85]:
df.head()

,oid,category,text,clean_text
0,365271984,0,Волшебные фото Виктория Поплавская Евгения Мед...,волшебные фото виктория поплавская евгения мед...
1,503385563,1,Возвращение подземелье Треша Эйфория тупости ж...,возвращение подземелье треша эйфория тупость ж...
2,146016084,2,Лучшие чешские вратари Доминик Доминатор Гашек...,хороший чешский вратарь доминик доминатор гаше...
3,933865449,3,Rtokenoid валрак решил нас подкормить сильно с...,валрак решить подкормить сильно свежий слух им...
4,713550145,4,Шестеркин затаскивает Рейнджерс финал Восточно...,шестеркин затаскивать рейнджерс финал восточны...


In [87]:
X_text = df['clean_text']
y = df['category']

In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import SGDClassifier 

vectorizer = TfidfVectorizer()

In [132]:
vectors = vectorizer.fit_transform(X_text)

In [93]:
X = np.asarray(vectors.todense())

In [102]:
X_train, X_test, y_train, y_test= train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

In [95]:
X_train.shape

(2905, 89786)

In [97]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [98]:
clf = SGDClassifier(max_iter=1000, warm_start=True, tol=1e-8)

In [113]:
# clf = SVC(verbose=2)

In [99]:
clf.fit(X_train, y_train)

SGDClassifier(tol=1e-08, warm_start=True)

In [108]:
def print_res(y_test, answers_pred):
    print(f'Accuracy: {accuracy_score(y_test, answers_pred)}')
    print(f'Precision: {precision_score(y_test, answers_pred, average="micro")}')
    print(f'Recall: {recall_score(y_test, answers_pred,average="micro")}')
    print(f'F1: {f1_score(y_test, answers_pred,average="micro")}')
    
pred = clf.predict(X_test)
print_res(y_test, pred)

Accuracy: 0.9896800825593395
Precision: 0.9896800825593395
Recall: 0.9896800825593395
F1: 0.9896800825593395


In [118]:
clf.fit(X, y)

SGDClassifier(tol=1e-08, warm_start=True)

In [121]:
test = pd.read_csv('test.csv')
test.head()

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...


In [122]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26260 entries, 0 to 26259
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   oid     26260 non-null  int64 
 1   text    26260 non-null  object
dtypes: int64(1), object(1)
memory usage: 410.4+ KB


In [123]:
test['text'] = test.groupby(['oid'])['text'].transform(lambda x: ' '.join(x))
test.drop_duplicates(inplace=True)
test.shape

(2626, 2)

In [124]:
test.reset_index(inplace=True)

In [125]:
test['text']=preprocessing(test['text'])

In [126]:
tokens = test['text'].apply(tokenize)
for i in range(len(tokens)):
    tokens[i] = remove_token_words(tokens[i])
for i in range(len(tokens)):
    tokens[i] = lower_text(tokens[i])
docs = []
for i in tokens:
    docs.append(" ".join(i))
test['clean_text'] = docs

#removing stopwords
test['clean_text'] = test['clean_text'].apply(remove_stopwords)
#lemmatization
test['clean_text'] = test['clean_text'].apply(lemmatize)

In [129]:
test.head()

,oid,text,clean_text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ПОКУПКОЙ ПИШИТЕ ВС...,спочно сообщество продаваться покупка пишите г...
1,452466036,Естественное восстановление после тяжелой трен...,естественный восстановление тяжёлый тренировка...
2,161038103,Тема нарядов продолжается Одна британских журн...,тема наряд продолжаться одна британский журнал...
3,663621910,Привет Избранный спрашиваешь себя ЧТО здесь пр...,привет избрать спрашиваешь происходить ответ с...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ ДЕЛЕ Андрей Рублев успешно...,король пятисотников деле андрей рублёв успешно...


In [133]:
test_vectors = vectorizer.transform(test['clean_text'])

In [134]:
test_pred = clf.predict(test_vectors)

In [142]:
res_dict = dict(map(lambda i,j : (i,j) , test.oid.values,test_pred))

In [152]:
submit = pd.read_csv('test.csv')

In [168]:
test['category'] = test_pred

In [172]:
test.replace({'category':inv_cat}, inplace= True)

In [182]:
category = []
for i in submit.oid.values:
    category.append(d[i])

In [183]:
submit['category'] = category